# Démonstration - filtrage gaussien vs. filtre "en boîte"

In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
# mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
# mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=20)

import warnings
warnings.filterwarnings("ignore")

# import necessarry libs
from imageio import imread
import numpy as np
from scipy.signal import convolve2d as conv2
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy.fftpack import fft2, fftshift, ifft2

# define gaussian filter
def f_gauss(size, sigma):
    # gaussian filter
    x, y = np.mgrid[-size[0]//2 + 1:size[0]//2 + 1, -size[1]//2 + 1:size[1]//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

# define average filter
def f_average(size):
    k = np.ones(size)
    return k/k.sum()

In [ ]:
# Lire l'image    
im = imread('img2.jpg').astype('float')/255.0;
im = rgb2gray(im)
im = resize(im, [400, 600]);

In [ ]:
fftsize = 512;
# FFT de l'image
im_fft = fft2(im, [fftsize, fftsize])

In [ ]:
hs = 30;
fil = f_gauss([hs*2+1, hs*2+1], 2);
# hs = 5;
# fil = f_average([hs*2+1, hs*2+1]);

In [ ]:
# FFT du filtre (même dimensions que l'image)
fil_fft = fft2(fil, [fftsize, fftsize]); 

In [ ]:
# Affichons le tout
# %matplotlib notebook
fig = plt.figure(figsize=(10,5), num='image')
ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(im); ax1.axis('off'); ax1.set_title('image')
ax2 = fig.add_subplot(1, 2, 2)
fmap = ax2.imshow(fil); ax2.axis('off'); ax2.set_title('filtre');
fig.colorbar(fmap, ax=ax2,fraction=0.046, pad=0.04)
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,5), num='FFT image')

ax1 = fig.add_subplot(1, 2, 1)
fmap = ax1.imshow(np.log(np.abs(fftshift(im_fft)))); 
ax1.axis('off'); ax1.set_title('FFT image')
fig.colorbar(fmap, ax=ax1,fraction=0.046, pad=0.04)

ax2 = fig.add_subplot(1, 2, 2)
fmap = ax2.imshow(np.log(np.abs(fftshift(fil_fft)))); 
ax2.axis('off'); ax2.set_title('FFT filtre');
fig.colorbar(fmap, ax=ax2,fraction=0.046, pad=0.04)
plt.tight_layout()

# Filtrons!

In [ ]:
# multiplication dans le domaine frequentiel
im_fil_fft = im_fft * fil_fft;

# prenons l'inverse
im_fil = ifft2(im_fil_fft).real; 

# enlever le "padding"
im_fil = im_fil[1+hs:im.shape[0]+hs, :]; 

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10,5), num='IFFT')

ax1 = fig.add_subplot(1, 2, 1)
fmap = ax1.imshow(im); 
ax1.axis('off'); ax1.set_title('Image originale')

ax2 = fig.add_subplot(1, 2, 2, sharex = ax1, sharey = ax1)
fmap = ax2.imshow(im_fil); 
ax2.axis('off'); ax2.set_title('Image filtrée');
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,5), num='fftshift')

ax1 = fig.add_subplot(1, 3, 1)
fmap = ax1.imshow(np.log(np.abs(fftshift(im_fft)))); 
ax1.axis('off'); ax1.set_title('FFT image')
fig.colorbar(fmap, ax=ax1,fraction=0.046, pad=0.04)

ax2 = fig.add_subplot(1, 3, 2, sharex = ax1, sharey = ax1)
fmap = ax2.imshow(np.log(np.abs(fftshift(fil_fft)))); 
ax2.axis('off'); ax2.set_title('FFT filtre');
fig.colorbar(fmap, ax=ax2,fraction=0.046, pad=0.04)

ax3 = fig.add_subplot(1, 3, 3, sharex = ax1, sharey = ax1)
fmap = ax3.imshow(np.log(np.abs(fftshift(im_fft*fil_fft)))); 
ax3.axis('off'); ax3.set_title('FFT image filtre');
fig.colorbar(fmap, ax=ax3,fraction=0.046, pad=0.04)
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(5,5), num='fftshift jet')
fmap = plt.imshow(np.log(np.abs(fftshift(im_fil_fft))), cmap='jet'); 
plt.axis('off'); 
plt.colorbar(fmap, fraction=0.046, pad=0.04);